In [ ]:
include(joinpath(dirname(Base.active_project()), "src", "AutoSpider.jl"))
using .AutoSpider, HDF5

include(joinpath(AutoSpider.FUNCTION_PATH, "VizUtils.jl"))
using .VizUtils

viz_dict::Dict = Dict()

orders::Array{Int} = [2,3,4,5,6]

for i in eachindex(orders)
    viz_dict[i] = Dict()
    path::String = joinpath(AutoSpider.HDF5_PATH, "orszag-tang", "64x64", "orszag-tang-RK" * repr(orders[i]) * ".hdf5")
    h5open(path, "r") do file
        viz_dict[i]["p"] = read(file, "result/value/p/value")
    end
end


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; 

In [ ]:
using Statistics, LinearAlgebra, GLMakie

tmax=10
tr = 1:tmax
tsqsum(x) = [mean(x[:,:,t].^2) for t = 1:tmax]

psums::Vector{Vector{Float64}} = []
for i in eachindex(orders)
    push!(psums, tsqsum(viz_dict[i]["p"]))
end

mins::Vector{Float64} = []
for t in 1:tmax
    push!(mins, mean(permutedims(hcat(psums...))[:,t]))
end

println(mins)

f = Figure()
ax = Axis(f[1, 1], yscale=log10)
lines!(ax, tr, abs.(psums[1] - mins) .+ 1e-6, label="RK2")
lines!(ax, tr, abs.(psums[2] - mins) .+ 1e-6, label="RK3")
lines!(ax, tr, abs.(psums[3] - mins) .+ 1e-6, label="RK4")
lines!(ax, tr, abs.(psums[4] - mins) .+ 1e-6, label="RK5")
lines!(ax, tr, abs.(psums[5] - mins) .+ 1e-6, label="RK6", linewidth=3, linestyle = :dash)

axislegend(position = :lt)

f

In [ ]:
model_results::Dict = Dict()

for i in eachindex(orders)
    path::String = joinpath(AutoSpider.HDF5_PATH, "orszag-tang", "64x64", "orszag-tang-RK" * repr(orders[i]) * ".hdf5")
    model_results[i] = AutoSpider.auto_spider(path, joinpath("compare", "settings_64.json"))
    println()
end

In [ ]:
# u momentum equation coefficient differences

dom = 1:maximum([length(model_results[i][5]["coeff"]) for i in eachindex(orders)])

f = Figure()
ax = Axis(f[1, 1], yscale=log10)

ideal = [1., 1., 1., -0.01, -0.01, 1., -1., -1.]

scatter!(ax, dom, (ideal - model_results[1][2]["coeff"]).^2 .+ 1e-12, label="RK2")
scatter!(ax, dom, (ideal - model_results[2][5]["coeff"]).^2 .+ 1e-12, label="RK3")
scatter!(ax, dom, (ideal - model_results[3][5]["coeff"]).^2 .+ 1e-12, label="RK4")
scatter!(ax, dom, (ideal - model_results[4][5]["coeff"]).^2 .+ 1e-12, label="RK5")
scatter!(ax, dom, (ideal - model_results[5][5]["coeff"]).^2 .+ 1e-12, label="RK6")

axislegend(position = :rb)
f

In [ ]:
for j in eachindex(orders)
    for i = 1:5
        println(model_results[j][i]["coeff"])
    end
    println()
end